In [1]:
import collections
import csv
import sys
import time
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
chrome_options = Options()
chrome_prefs = {'profile.default_content_setting_values': {'images': 2, 'javascript': 2}}
chrome_options.add_experimental_option('prefs', chrome_prefs)
chrome_options.add_argument('-headless')
chrome_options.add_argument('-no-sandbox')
chrome_options.add_argument('-disable-dev-shm-usage')
chrome_options.add_argument("enable-automation")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--dns-prefetch-disable")
chrome_options.add_argument("--disable-gpu")

In [3]:
driver_path = ChromeDriverManager().install()
service = Service(executable_path=driver_path)

# Création de l'instance du navigateur
driver = webdriver.Chrome(options=chrome_options, service=service)

In [3]:
url = "https://fr.shein.com/Women-Clothing-sc-017172961.html?adp=&categoryJump=true&ici=fr_tab04navbar04&src_identifier=fc%3DWomen%20Clothing%60sc%3DWomen%20Clothing%60tc%3D0%60oc%3D0%60ps%3Dtab04navbar04%60jc%3DitemPicking_017172961&src_module=topcat&src_tab_page_id=page_real_class1704888170646"
#url = "https://fr.shein.com/Men-Clothing-c-2026.html?ici=fr_tab06navbar04&scici=navbar_MenHomePage~~tab06navbar04~~4~~real_2026~~~~0&src_module=topcat&src_tab_page_id=page_home1673351679865&src_identifier=fc%3DMen%60sc%3DV%C3%8ATEMENTS%60tc%3D0%60oc%3D0%60ps%3Dtab06navbar04%60jc%3Dreal_2026&srctype=category&userpath=category-V%C3%8ATEMENTS&child_cat_id=1970"


Version optimisé

In [11]:
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

# Accéder à la page
url = "https://fr.shein.com/Women-Clothing-sc-017172961.html?..."
driver.get(url)
time.sleep(10)
# Fonction pour extraire le texte d'un élément, avec texte par défaut si non trouvé
def extract_text(elements, default_text):
    return elements[0].text if elements else default_text

for _ in range(10):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)  # Simule la touche "Page Down"
    time.sleep(0.1)

# Trouver tous les éléments de produit
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product-card"))
)

product_elements = driver.find_elements(By.CSS_SELECTOR, ".product-card")

# Extraction des données des produits
products = []
for product in product_elements:
    name = extract_text(product.find_elements(By.CSS_SELECTOR, "a.goods-title-link--jump"), "Nom inconnu")
    price = extract_text(product.find_elements(By.CSS_SELECTOR, "span.normal-price-ctn__sale-price"), "Prix inconnu")
    promotion = extract_text(product.find_elements(By.CSS_SELECTOR, "div.product-card__discount-label"), "Pas de promotion")
    
    sales_elements = product.find_elements(By.CSS_SELECTOR, "p.product-card__selling-proposition-text")
    sales_text = extract_text(sales_elements, "").replace('k', '000')
    match = re.search(r'\d+', sales_text)
    sales_number = int(match.group()) if match else 0

    products.append((name, price, promotion, sales_number))

# Fermer le WebDriver
driver.quit()

# Afficher les résultats
for name, price, promotion, sales in products:
    print(f"{name} - {price} - {promotion} - {sales}")

print(f"Nombre de produits: {len(products)}")


SHEIN Najma Abaya manches trompette (sans ceinture) - 18,99€ - -3% - 0
Sweat-shirt Imprimé Cœur À Doublure Thermique - 8,49€ - -15% - 1
Robe de soirée asymétrique fendu à paillettes - 31,99€ - -9% - 0
Yoga Basic Legging De Sport Unicolore Taille Haute À Poche De Téléphone - 9,99€ - Pas de promotion - 0
SHEIN BASICS Legging de grossesse taille haute unicolore - 6,39€ - -20% - 6
SHEIN Privé Jupe Midi À Ourlet À Volants Multicouche Pour Femme - 14,99€ - Pas de promotion - 0
T-shirt en dentelle - 7,24€ - -31% - 5
Ensemble De Pyjama Passepoil Contrasté En Satin - 10,99€ - Pas de promotion - 3
Dazy Star T-shirt Unicolore Col Rond Ajusté Thermique - 5,93€ - -34% - 0
SHEIN LUNE Pull À Blocs De Couleurs Épaule tombante - 14,87€ - -15% - 0
SHEIN Mulvari Kurta à imprimé baroque fendu & pantalon - 23,36€ - -18% - 0
Haut Imprimé En Maille À Double Couche Pour Femme À Manches 3/4 - 12,49€ - -7% - 0
VUTRU Veste De Sport Femme Ajusté Avec Trou De Pouce - 14,99€ - Pas de promotion - 0
SHEIN LUNE Pantal

Version améliorée pour le scrapping : Intègre un délai de 10-15 secondes entre les requêtes pour éviter un bannissement et assurer le chargement complet des produits. 
Ajoute une simulation de défilement de page pour charger tous les produits.
Malgré ces améliorations, certains articles peuvent ne pas se charger correctement. 
Cette approche vise à optimiser la récupération des données tout en respectant les politiques du site.


In [28]:
# Configuration du WebDriver
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)
page = 33
#products = []
# Formatage de l'URL avec la valeur de page



while page != 32: 
    url = f"https://fr.shein.com/Women-Clothing-sc-017172961.html?page={page}"
    driver.get(url)
    time.sleep(10)
    def extract_text(elements, default_text):
        return elements[0].text if elements else default_text
    for _ in range(15):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 
        time.sleep(0.1)


    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product-card"))
    )
    product_elements = driver.find_elements(By.CSS_SELECTOR, ".product-card") 
    for product in product_elements:
        name = extract_text(product.find_elements(By.CSS_SELECTOR, "a.goods-title-link--jump"), "Nom inconnu")
        price = extract_text(product.find_elements(By.CSS_SELECTOR, "span.normal-price-ctn__sale-price"), "Prix inconnu")
        promotion = extract_text(product.find_elements(By.CSS_SELECTOR, "div.product-card__discount-label"), "Pas de promotion")
        
        sales_elements = product.find_elements(By.CSS_SELECTOR, "p.product-card__selling-proposition-text")
        sales_text = extract_text(sales_elements, "").replace('k', '000')
        match = re.search(r'\d+', sales_text)
        sales_number = int(match.group()) if match else 0

        products.append((name, price, promotion, sales_number))
    print(f"Nombre de produits: {len(products)}")
    page = page + 1
    


driver.quit()




Nombre de produits: 2620
Nombre de produits: 2737
Nombre de produits: 2854
Nombre de produits: 2966
Nombre de produits: 3074
Nombre de produits: 3179
Nombre de produits: 3284
Nombre de produits: 3382
Nombre de produits: 3457


TimeoutException: Message: 


In [29]:
# Afficher les résultats
print(products)
print(len(products))
#print(len(price))
#print(len(promotion))
#print(len(sales_number))


[('', '', '', 0), ('', '', '', 0), ('', '', '', 0), ('', '', 'Pas de promotion', 0), ('', '', '', 0), ('', '', 'Pas de promotion', 0), ('', '', '', 0), ('', '', 'Pas de promotion', 0), ('', '', '', 0), ('', '', '', 0), ('', '', '', 0), ('', '', 'Pas de promotion', 0), ('', '', 'Pas de promotion', 0), ('', '', '', 0), ('', '', '', 0), ('', '', 'Pas de promotion', 0), ('', '', '', 0), ('', '', 'Pas de promotion', 0), ('', '', 'Pas de promotion', 0), ('', '', '', 0), ('', '', '', 0), ('', '', '', 0), ('', '', 'Pas de promotion', 0), ('', '', '', 0), ('', '', 'Pas de promotion', 0), ('', '', 'Pas de promotion', 0), ('', '', 'Pas de promotion', 0), ('', '', 'Pas de promotion', 0), ('', '', '', 0), ('', '', '', 0), ('', '', 'Pas de promotion', 0), ('', '', '', 0), ('', '', '', 0), ('', '', 'Pas de promotion', 0), ('', '', 'Pas de promotion', 0), ('', '', 'Pas de promotion', 0), ('', '', '', 0), ('', '', '', 0), ('', '', 'Pas de promotion', 0), ('', '', 'Pas de promotion', 0), ('', '', 'Pas d

In [24]:
with open('products.csv', 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Écrire l'en-tête du CSV
    csv_writer.writerow(['Nom du produit', 'Prix', 'Promotion', 'Ventes'])

    # Écrire les données des produits dans le CSV
    for name, price, promotion, sales in products:
        csv_writer.writerow([name, price, promotion, sales])